In [1]:
from datasets import load_dataset

# data = load_dataset("nielsr/docvqa_1200_examples_donut")
# data = load_dataset("GBaker/MedQA-USMLE-4-options")
data =  load_dataset("tau/commonsense_qa")

train_df = data['train'].to_pandas()
# test_df = data['test'].to_pandas()
test_df = data['validation'].to_pandas()
test_df.head()

,id,question,question_concept,choices,answerKey
0,1afa02df02c908a558b4036e80242fac,A revolving door is convenient for two directi...,revolving door,"{'label': ['A', 'B', 'C', 'D', 'E'], 'text': [...",A
1,a7ab086045575bb497933726e4e6ad28,What do people aim to do at work?,people,"{'label': ['A', 'B', 'C', 'D', 'E'], 'text': [...",A
2,b8c0a4703079cf661d7261a60a1bcbff,Where would you find magazines along side many...,magazines,"{'label': ['A', 'B', 'C', 'D', 'E'], 'text': [...",B
3,e68fb2448fd74e402aae9982aa76e527,Where are you likely to find a hamburger?,hamburger,"{'label': ['A', 'B', 'C', 'D', 'E'], 'text': [...",A
4,2435de612dd69f2012b9e40d6af4ce38,James was looking for a good place to buy farm...,farmland,"{'label': ['A', 'B', 'C', 'D', 'E'], 'text': [...",A


In [2]:
sample = test_df.head(1)
for col in sample.columns:
    print(f"{col}: {sample[col].values[0]}")

id: 1afa02df02c908a558b4036e80242fac
question: A revolving door is convenient for two direction travel, but it also serves as a security measure at a what?
question_concept: revolving door
choices: {'label': array(['A', 'B', 'C', 'D', 'E'], dtype=object), 'text': array(['bank', 'library', 'department store', 'mall', 'new york'],
      dtype=object)}
answerKey: A


In [5]:
from evaluate import load
import pandas as pd
perplexity = load("perplexity", module_type="metric")
predictions = []
for i in range(10):
    predictions.append(test_df.loc[i]["question"])
predictions.append(" ")
results = perplexity.compute(predictions=predictions, model_id='gpt2')
print(results)
# tmp_df = pd.DataFrame({"Predictions": predictions, "Perplexity": results['perplexities']})
# tmp_df.add(
# lowest_perplexity_idx = tmp_df["Perplexity"].idxmin()
# reference_question = tmp_df.loc[lowest_perplexity_idx]["Predictions"]
# reference_question

  0%|          | 0/1 [00:00<?, ?it/s]

{'perplexities': [102.255615234375, 36.754398345947266, 194.2383270263672, 142.31822204589844, 123.75379180908203, 607.9295654296875, 77.58659362792969, 48.806671142578125, 338.6368408203125, 72.5635986328125, 12869.53125], 'mean_perplexity': 1328.5795340104537}


In [3]:
import numpy as np
import pandas as pd


def create_choices(df, n_choices):
    new_df = df.copy()
    rows = []
    for _, row in df.iterrows():
        choices = row['choices']
        answer_label = row['answerKey']
        answer_text = choices['text'][np.where(choices['label'] == answer_label)[0][0]]
        
        # 保证选项中包含答案
        other_choices = [(label, text) for label, text in zip(choices['label'], choices['text']) if label != answer_label]
        selected_other_choices = np.random.choice(len(other_choices), n_choices - 1, replace=False)
        selected_choices = [other_choices[i] for i in selected_other_choices]
        
        # 构建最终选项列表
        final_choices = [(answer_label, answer_text)] + selected_choices
        np.random.shuffle(final_choices)  # 打乱顺序
        
        # 重新构建
        new_choices = {
            'label': np.array([c[0] for c in final_choices]),
            'text': np.array([c[1] for c in final_choices])
        }
        
        # 新行
        rows.append({
            "id": row['id'],
            "question": row['question'],
            "question_concept": row['question_concept'],
            "choices": new_choices,
            "answerKey": row['answerKey']
        })
    return pd.DataFrame(rows)

In [4]:
df_2_choices = create_choices(test_df, 2)
df_3_choices = create_choices(test_df, 3)
df_4_choices = create_choices(test_df, 4)

df_2_choices.to_json("dataset/commonsense_qa_2_choices.json", orient='records', lines=True)
df_3_choices.to_json("dataset/commonsense_qa_3_choices.json", orient='records', lines=True)
df_4_choices.to_json("dataset/commonsense_qa_4_choices.json", orient='records', lines=True)

In [4]:
import pandas as pd
# df_2_choices = pd.read_json("dataset/commonsense_qa_2_choices.json", orient='records', lines=True)
df_2_choices = pd.read_json("results/VQA_res/GPT35_JSON/paraphrased_questions_T0.25.json", orient='records')
df_2_choices.shape

(200, 14)

In [33]:
test_df = test_df.sample(10).reset_index(drop=True)
test_df.head()

,id,question,question_concept,choices,answerKey
0,5279a2ea333ba8a5bf3a7637a7279da1,Batman bought beer. There were no bottles ava...,beer,"{'label': ['A', 'B', 'C', 'D', 'E'], 'text': [...",D
1,fb64149cf01c5b496d986f56852273e9,What is a place that has large cable hanging o...,cable,"{'label': ['A', 'B', 'C', 'D', 'E'], 'text': [...",D
2,2a32b1e541b1daae04690d0d3a4b3310,The pitcher felt stress and tension on the mou...,mound,"{'label': ['A', 'B', 'C', 'D', 'E'], 'text': [...",C
3,df3d27338bcf86b341b8b02d4309daf5,Where do you keep your pizza slice before you ...,pizza,"{'label': ['A', 'B', 'C', 'D', 'E'], 'text': [...",B
4,5496c7293f653120e5a5213db2d7b103,Where is beer drank by people watching sticks ...,beer,"{'label': ['A', 'B', 'C', 'D', 'E'], 'text': [...",C


In [35]:
# Check train_df['query']['en'] contains \n
# test_df['question'].replace('\n', ', ', regex=True, inplace=True)

for i in range(len(test_df)):
    if '\n' in test_df.loc[i]['question']:
        print(i)
        print(test_df.loc[i]['question'])
        
# print(test_df.loc[2]['question'])

In [40]:
prompt = ""
for idx in range(len(test_df.loc[2]['choices']['label'])):
    prompt += test_df.loc[2]['choices']['label'][idx] + ". " + test_df.loc[2]['choices']['text'][idx] + '\n'
prompt = test_df.loc[2]['question'] + '\nAnswer the question with the following options: ' + '\n' + prompt + 'Answer Index: '
print(prompt)

The pitcher felt stress and tension on the mound, what did he feel like?
Answer the question with the following options: 
A. desert
B. baseball field
C. hell
D. baseball diamond
E. baseball stadium
Answer Index: 


In [41]:
from utils import append2file
from openai_backTranslation import generate
import evaluate
import pandas as pd
import numpy as np

def convert_ABCD(x : list):
    x = pd.Series(x)
    x.fillna("C", inplace=True)
    return x.map(
        {
            "A": 0,
            "B": 1,
            "C": 2,
            "D": 3,
            "E": 4
        }
    )


acc_metric = evaluate.load("accuracy")
path_prefix = "results/"
res_path = f"{path_prefix}res_csqa_GT.txt"
logits_bias_dict = {"32": 100, "33": 100, "34": 100, "35": 100, "36": 100}     # A-E: 32-36
test_df = test_df.head(10)
gt_answer = test_df["answerKey"].tolist()
gt_answer = convert_ABCD(gt_answer)
acc_scores = []


append2file(res_path, f"Answer paraphrased GT:")
for _ in range(5):
    T_predictions = []
    for i in range(len(test_df)):
        prompt = ""
        for idx in range(len(test_df.loc[i]['choices']['label'])):
            prompt += test_df.loc[i]['choices']['label'][idx] + ". " + test_df.loc[i]['choices']['text'][idx] + '\n'
        prompt = test_df.loc[i]['question'] + '\nAnswer the question with the following options: ' + '\n' + prompt + 'Answer Index: '
        
        T_predictions.append(generate(prompt, temperature=0.0, logits_dict=logits_bias_dict, max_tokens=1))
    T_predictions = convert_ABCD(T_predictions)
    acc_score = acc_metric.compute(references=gt_answer, predictions=T_predictions)
    acc_scores.append(acc_score['accuracy'])
append2file(res_path, f"Accuracy Mean: {np.mean(acc_scores)}; Std: {np.std(acc_scores)}")

Batman bought beer.  There were no bottles available.  He had to settle for what?.
Answer the question with the following options: 
A. shelf
B. soccer game
C. keg
D. can
E. refrigerator
Answer Index: 
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
D
What is a place that has large cable hanging overhead?
Answer the question with the following options: 
A. radio shack
B. electrical device
C. shower
D. substation
E. television
Answer Index: 
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
D
The pitcher felt stress and tension on the mound, what did he feel like?
Answer the question with the following options: 
A. desert
B. baseball field
C. hell
D. baseball diamond
E. baseball stadium
Answer Index: 
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
C
Where do you keep your pizza slice before you eat it?
Answer the question with the following options: 
A. table
B. plate
C. restaurant
D. oven
E. popular
Answer Index: 
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
B
Where is beer drank by people watching sticks and pu

In [ ]:
import evaluate
import pandas as pd
acc_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def convert_ABCD(x : list):
    x = pd.Series(x)
    return x.map(
        {
            "A": 0,
            "B": 1,
            "C": 2,
            "D": 3,
        }
    )
    
predications = ['A', 'B', 'C', 'D', 'A']
references = ['A', 'A', 'A', 'A', 'A']
print(convert_ABCD(test_df["answer_idx"].tolist()))
acc_metric.compute(predictions=convert_ABCD(predications), references=convert_ABCD(references))

0       1
1       3
2       1
3       3
4       1
       ..
1268    3
1269    3
1270    1
1271    2
1272    2
Length: 1273, dtype: int64


{'accuracy': 0.4}

In [13]:
from openai_backTranslation import generate
import pandas as pd


paraphrased_df = pd.DataFrame()
temperture_list = [0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5]
t = 1.0
for tempreture in temperture_list:
    # paraphrased_df[f'T_{tempreture}'] = test_df['query'].apply(lambda x: generate(f"Document : {x['en']}\nParaphrase of the document :", tempreture, stop=["\n"]))
    paraphrased_df[f'T_{tempreture}'] = test_df['query'].apply(lambda x: generate(f"Question : {x['en']}\nParaphrase of the question :", temperature=t, stop=["\n"]))

Question : How much is the 'Excess of expenditures over income' ?
Paraphrase of the question :
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
 What was the loss for the year ?
Question : How much is the amount from 'Trusts' in $?
Paraphrase of the question :
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
 What is the total of 'Trusts' in $?
Question : What is the brand name of the heritage leisure segment of Hotels?
Paraphrase of the question :
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
 What is the name of the branding for the Heritage Hotels?
Question : How much is the total income ?
Paraphrase of the question :
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
 What's the sum of all income?
Question : how much order is to be shipped to hong kong
Paraphrase of the question :
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
 find the number of ordres shipped to hong kong.
Question : How many nomination committee meetings has S. Banerjee attended?
Paraphrase of the question :
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

In [14]:
import evaluate
import numpy as np
rouge_metric = evaluate.load("rouge")
bleu_metric = evaluate.load("bleu")
references = test_df['query'].apply(lambda x: x['en']).tolist()
rouge1_scores, rouge2_scores, rougeL_scores = [], [], []
bleu_scores = []
for tempreture in temperture_list:
    predictions = paraphrased_df[f'T_{tempreture}'].tolist()
    score = rouge_metric.compute(references=references, predictions=predictions)
    # print(f"tempreture: {tempreture};   rouge1: {score['rouge1']}; rouge2: {score['rouge2']}; rougeL: {score['rougeL']}")
    print(f"rouge1: {score['rouge1']}; rouge2: {score['rouge2']}; rougeL: {score['rougeL']}")
    rouge1_scores.append(score['rouge1'])
    rouge2_scores.append(score['rouge2'])
    rougeL_scores.append(score['rougeL'])
    bleu_score = bleu_metric.compute(references=references, predictions=predictions)
    bleu_scores.append(bleu_score['bleu'])
    print(f"bleu: {bleu_score}")
print(f"rouge1 mean: {np.mean(rouge1_scores)}; rouge2 mean: {np.mean(rouge2_scores)}; rougeL mean: {np.mean(rougeL_scores)}")
print(f"rouge1 std: {np.std(rouge1_scores)}; rouge2 std: {np.std(rouge2_scores)}; rougeL std: {np.std(rougeL_scores)}")
print(f"bleu mean: {np.mean(bleu_scores)}")
print(f"bleu std: {np.std(bleu_scores)}")

rouge1: 0.5965671847222191; rouge2: 0.42886289947610234; rougeL: 0.5891202437912965
bleu: {'bleu': 0.4149889959726979, 'precisions': [0.6296296296296297, 0.47959183673469385, 0.375, 0.28205128205128205], 'brevity_penalty': 0.9816518956846597, 'length_ratio': 0.9818181818181818, 'translation_length': 108, 'reference_length': 110}
rouge1: 0.610825583085645; rouge2: 0.3861214310285518; rougeL: 0.5511990331649774
bleu: {'bleu': 0.3094643885681913, 'precisions': [0.6095238095238096, 0.3684210526315789, 0.24705882352941178, 0.2], 'brevity_penalty': 0.9534969548334766, 'length_ratio': 0.9545454545454546, 'translation_length': 105, 'reference_length': 110}
rouge1: 0.5742439967439967; rouge2: 0.33661510419405155; rougeL: 0.5191732804232805
bleu: {'bleu': 0.22862823756510595, 'precisions': [0.5289256198347108, 0.27927927927927926, 0.16831683168316833, 0.10989010989010989], 'brevity_penalty': 1.0, 'length_ratio': 1.1, 'translation_length': 121, 'reference_length': 110}
rouge1: 0.5492139903440212;

In [15]:
gt_answer = []
for i in range(len(test_df)):
    answer = ""
    for word in test_df.loc[i]['answers']:
        answer += word + ", "
    answer = answer[:-2]
    gt_answer.append(answer)
print(gt_answer)

temperture_list = [0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5]
rouge1_scores = []
rouge2_scores = []
rougeL_scores = []
bleu_scores = []
repeats = 5
import numpy as np

# for t in temperture_list:
#     T_rouge1 = []
#     T_rouge2 = []
#     T_rougeL = []
#     for _ in range(repeats):
#         T_predictions = []
#         for i in range(len(test_df)):
#             prompt = ""
#             for word in test_df.loc[i]['words']:
#                 prompt += word + ", "
#             prompt = "Extracted OCR tokens from image:\n" + prompt[:-2] + "\nQuestion: " + paraphrased_df.loc[i][f'T_{t}'] + "\nAnswer the question with short term:"
#             T_predictions.append(generate(prompt, temperature=0.0, stop=["\n"]))
#         score = rouge_metric.compute(references=gt_answer, predictions=T_predictions)
#         T_rouge1.append(score['rouge1'])
#         T_rouge2.append(score['rouge2'])
#         T_rougeL.append(score['rougeL'])
#     rouge1_scores.append({"mean": np.mean(T_rouge1), "std": np.std(T_rouge1)})
#     rouge2_scores.append({"mean": np.mean(T_rouge2), "std": np.std(T_rouge2)})
#     rougeL_scores.append({"mean": np.mean(T_rougeL), "std": np.std(T_rougeL)})
    
    
for t in temperture_list:
    T_predictions = []
    for i in range(len(test_df)):
        prompt = ""
        for word in test_df.loc[i]['words']:
            prompt += word + ", "
        prompt = "Extracted OCR tokens from image:\n" + prompt[:-2] + "\nQuestion: " + paraphrased_df.loc[i][f'T_{t}'] + "\nAnswer the question with short term:"
        T_predictions.append(generate(prompt, temperature=0.0, stop=["\n"]))
    score = rouge_metric.compute(references=gt_answer, predictions=T_predictions)
    rouge1_scores.append(score['rouge1'])
    rouge2_scores.append(score['rouge2'])
    rougeL_scores.append(score['rougeL'])
    bleu_score = bleu_metric.compute(references=gt_answer, predictions=T_predictions)
    bleu_scores.append(bleu_score['bleu'])

['$ 3,038,444, 3,038,444', '$ 7,265,516, 7,265,516', 'WelcomHeritage', '8,899,947', '18 million, 18 million order, the 18 million order', '2', 'Emotional Enhancement', 'WelcomHotel, WelComHotel', 'mint-o Ultra mintz, mint-o Ultra Mintz', '6']
Extracted OCR tokens from image:
W., K., KELLOGG, FOUNDATION, INCOME, AND, EXPENDITURES, Fiscal, Year, Ended, August, 31,, 1963, Income, Trusts, $, 7,265,516, Investments, 1,634,431, 8,899,947, Expenditures, Program, payments, ., ., 11,369,878, Refunds, on, expenditures, in, prior, years, (42,262, ), Administration, ., ., 610,775, 11,938,391, Excess, of, expenditures, over, income, ., ., $, 3,038,444, Source:, https://www.industrydocuments.ucsf.edu/docs/yshb0228
Question:  What was the loss for the year ?
Answer the question with short term:
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
  $3,038,444
Extracted OCR tokens from image:
W., K., KELLOGG, FOUNDATION, INCOME, AND, EXPENDITURES, Fiscal, Year, Ended, August, 31,, 1963, Income, Trusts, $, 7,265,5

In [16]:
for i in range(len(temperture_list)):
    print(f"rouge1: {rouge1_scores[i]}; rouge2: {rouge2_scores[i]}; rougeL: {rougeL_scores[i]}")
    # print(f"tempreture: {temperture_list[i]};   rouge1: {rouge1_scores[i]}; rouge2: {rouge2_scores[i]}; rougeL: {rougeL_scores[i]}")
print(f"rouge1 mean: {np.mean(rouge1_scores)}; rouge2 mean: {np.mean(rouge2_scores)}; rougeL mean: {np.mean(rougeL_scores)}")
print(f"rouge1 std: {np.std(rouge1_scores)}; rouge2 std: {np.std(rouge2_scores)}; rougeL std: {np.std(rougeL_scores)}")
print(f"bleu mean: {np.mean(bleu_scores)}")
print(f"bleu std: {np.std(bleu_scores)}")

rouge1: 0.5227272727272727; rouge2: 0.26650793650793647; rougeL: 0.5196969696969698
rouge1: 0.5696969696969697; rouge2: 0.2365079365079365; rougeL: 0.5757575757575758
rouge1: 0.43636363636363634; rouge2: 0.23253968253968255; rougeL: 0.43636363636363634
rouge1: 0.5363636363636364; rouge2: 0.3285714285714286; rougeL: 0.5363636363636364
rouge1: 0.6363636363636364; rouge2: 0.2365079365079365; rougeL: 0.6666666666666666
rouge1: 0.6; rouge2: 0.21428571428571433; rougeL: 0.6
rouge1: 0.4696969696969697; rouge2: 0.07936507936507937; rougeL: 0.4696969696969697
rouge1: 0.7393939393939394; rouge2: 0.3285714285714286; rougeL: 0.7393939393939395
rouge1: 0.6363636363636365; rouge2: 0.3285714285714286; rougeL: 0.6363636363636365
rouge1: 0.6363636363636364; rouge2: 0.2365079365079365; rougeL: 0.6666666666666666
rouge1: 0.4696969696969697; rouge2: 0.07936507936507937; rougeL: 0.47272727272727266
rouge1 mean: 0.5684573002754821; rouge2 mean: 0.2333910533910534; rougeL mean: 0.5745179063360882
rouge1 std:

In [17]:
# Concat the paraphrased questions with the original questions and answers
paraphrased_df['original_question'] = test_df['query'].apply(lambda x: x['en'])
paraphrased_df['original_answer'] = test_df['answers']
paraphrased_df['words'] = test_df['words']

# Save the paraphrased questions
# paraphrased_df.to_csv("paraphrased_questions_T10.csv", index=False)
paraphrased_df.to_json("paraphrased_questions_T10.json", orient='records')

In [18]:
import pandas as pd
import ast
import re

def str_to_list(cell):
    if pd.isna(cell) or not isinstance(cell, str):
        return []
    
    # 去除多余的空格和换行符
    cell = cell.replace('\n', ' ')
    
    # 添加逗号以正确分隔元素
    cell = re.sub(r"'\s+'", "', '", cell)
    
    # 尝试将字符串解析为列表
    try:
        # 使用 ast.literal_eval 将字符串解析为 Python 对象
        parsed_list = ast.literal_eval(cell)
        
        # 确保解析结果为列表
        if isinstance(parsed_list, list):
            # 去掉每个元素的单引号（如果存在）
            return [item.strip("'") for item in parsed_list]
    except (ValueError, SyntaxError):
        pass
    return []

df_path = "paraphrased_questions_T10.json"
# paraphrased_df = pd.read_csv(df_path, converters={'original_answer': str_to_list, 'words': str_to_list})
paraphrased_df = pd.read_json(df_path)
paraphrased_df.head()

,T_0.5,T_0.6,T_0.7,T_0.8,T_0.9,T_1.0,T_1.1,T_1.2,T_1.3,T_1.4,T_1.5,original_question,original_answer,words
0,What was the loss for the year ?,"What is the amount of negative value for ""Rev...",What is the amount of the deficit?,What is the difference between income and exp...,What is the difference between expenditures a...,What is the difference between the company's ...,,What is the value of the expenses that the co...,What's its mismatch/difference between expens...,What is the amount of loss that is shown by t...,What is the difference between spending and i...,How much is the 'Excess of expenditures over i...,"[$ 3,038,444, 3,038,444]","[W., K., KELLOGG, FOUNDATION, INCOME, AND, EXP..."
1,What is the total of 'Trusts' in $?,What is the value in dollars included under ‘...,What is the monetary value of 'Trusts'?,Find the value in $ for which the 'Trusts' ('...,What is the amount in $ for 'Trusts'?,What is the amount assigned to 'Trusts'?,What is the value of the Trust variable?,Find 'trusts' value in dollars.,What are the total 'Trusts' deposits in $?,What is the number on the x-axis that corresp...,What is the net amount of money (in $) after ...,How much is the amount from 'Trusts' in $?,"[$ 7,265,516, 7,265,516]","[W., K., KELLOGG, FOUNDATION, INCOME, AND, EXP..."
2,What is the name of the branding for the Heri...,What is the hotels heritage leisure brand name?,What is the name of the company that has a he...,Hotels’ heritage leisure segment has what bra...,Hotels' heritage leisure segment brand name?,The question is asking for the brand name of ...,Under what brand name are the hotels operatin...,What is the name of the brand for Hotels that...,"What category of hotels does the brand name ""...",Which name represents the leisure area of Hot...,What brand names are used for heritage leisur...,What is the brand name of the heritage leisure...,[WelcomHeritage],"[TC, Limited, Report, of, the, Directors, impa..."
3,What's the sum of all income?,What is the total income?,What is the cost of all the sales?,What is the total income amount?,What is the total revenue received?,What is total annual income?,What is the sum of income ?,What is the sum of all the earnings ?,what is the total sum of incomes ?,What is the total amount of money the company...,Total cost for entire farm .,How much is the total income ?,"[8,899,947]","[W., K., KELLOGG, FOUNDATION, INCOME, AND, EXP..."
4,find the number of ordres shipped to hong kong.,what is the quantity of order to be send to h...,What is the shipping quantity to Hong Kong?,what is the total order that will be sent to ...,How much amount of the order is going to be s...,To which country the order of this quantity h...,How much does it cost to ship to hong kong,what is the amount of order that will be deli...,What is the number of products to be delivere...,what is the shipping cost to hong kong,tell us the quantity of the order to be shipp...,how much order is to be shipped to hong kong,"[18 million, 18 million order, the 18 million ...","[DOMESTIC, PRODUCT, DEVELOPMENT, (cont'd.), PO..."


In [19]:
questions_gt_list = paraphrased_df['original_question'].tolist()
answers_gt_list = paraphrased_df['original_answer'].tolist()
print(questions_gt_list)
print(answers_gt_list)

["How much is the 'Excess of expenditures over income' ?", "How much is the amount from 'Trusts' in $?", 'What is the brand name of the heritage leisure segment of Hotels?', 'How much is the total income ?', 'how much order is to be shipped to hong kong', 'How many nomination committee meetings has S. Banerjee attended?', 'What is the main title of this document?', 'What is the brand name of the five star category of hotels?', "Which is 'India's most premium, sugarfree power mints'?", 'What is the committee strength for the sustainability committee meeting held on 5th April, 2012?']
[['$ 3,038,444', '3,038,444'], ['$ 7,265,516', '7,265,516'], ['WelcomHeritage'], ['8,899,947'], ['18 million', '18 million order', 'the 18 million order'], ['2'], ['Emotional Enhancement'], ['WelcomHotel', 'WelComHotel'], ['mint-o Ultra mintz', 'mint-o Ultra Mintz'], ['6']]


In [3]:
# for each row fill nan
paraphrased_df = paraphrased_df.fillna(' ')

In [54]:
# privacy_params = {
#     'eps': 1.0,
    
# }

In [55]:
# data = [(('approval',), 8), (('banded',), 8), (('art',), 8), (('papers',), 7), 
#         (('status',), 6), (('diesing',), 5), (('What',), 5), (('The',), 4), 
#         (('document',), 3), (('asking',), 3), (('request',), 2), (('documents',), 2), 
#         (('update',), 2), (('printing',), 2), (('whether',), 2), (('current',), 1), 
#         (('progress',), 1), (('paper',), 1), (('enquiry',), 1), (('state',), 1), 
#         (('clearance',), 1), (('talks',), 1), (('asks',), 1), (('design',), 1), 
#         (('ask',), 1), (('bandedpapers',), 1), (('diesinbg',), 1), (('regarding',), 1), 
#         (('information',), 1), (('permission',), 1), (('stationery',), 1), 
#         (('designs',), 1), (('offices',), 1), (('present',), 1), (('declined',), 1), 
#         (('stamp',), 1), (('It',), 1), (('die',), 1), (('producing',), 1), 
#         (('using',), 1), (('unknown',), 1), (('granted',), 1), (('bracelets',), 1), 
#         (('rubber',), 1), (('kept',), 1), (('oustanding',), 1), (('inquiry',), 1), 
#         (('verb',), 1), (('formal',), 1), (('use',), 1)]


# import opendp.prelude as dp

In [4]:
from openai_backTranslation import generate
import evaluate
rouge_metric = evaluate.load("rouge")
bleu_metric = evaluate.load("bleu")
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.util import ngrams
stopword_set = set(stopwords.words('english'))
import string
import random


temperture_list = [0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5]
repeats = 5
bleu_q_scores = []
bleu_a_scores = []
rouge1_q_scores = []
rouge2_q_scores = []
rougeL_q_scores = []
rouge1_a_scores = []
rouge2_a_scores = []
rougeL_a_scores = []

for _ in range(repeats):
    questions_list = []
    answers_list = []
    for i in range(len(paraphrased_df)):
        # Count token freq
        all_tokens = {} # key: token, value: count
        for t in temperture_list:
            sentence = paraphrased_df.loc[i][f'T_{t}']
            tokens = nltk.word_tokenize(sentence)
            onegrams = set(ngrams(tokens, 1))
            # onegrams = set(onegrams)
            # making onegrams a set to avoid duplicate tokens
            for token in onegrams:
                # only add one gram for one sentence
                if token in all_tokens:
                    all_tokens[token] += 1
                else:
                    all_tokens[token] = 1
        print(f"All Tokens:  {all_tokens}")
        
        # ================ Add Noise Here ================
        all_tokens_sorted = sorted(all_tokens.items(), key=lambda x: x[1], reverse=True)
        print(f"All Sorted Tokens:  {all_tokens_sorted}")
        # ignore those non-words tokens
        filtered_tokens = {}
        for token, count in all_tokens_sorted:
            if not all(word in string.punctuation for word in token) and token[0] not in stopword_set:
                filtered_tokens[token] = count
        filtered_tokens_sorted = sorted(filtered_tokens.items(), key=lambda x: x[1], reverse=True)
        print(f"Filtered Sorted Tokens:  {filtered_tokens_sorted}")
        
        # Non-DP: find the count threshold where the count gap is the largest
        actually_upper_bound = 0
        count_threshold = 0 # you need to creat a list to store all difference between the counts
        for k in range(len(filtered_tokens_sorted)-1):
            #print(k,len(filtered_tokens_sorted)-2)
            if filtered_tokens_sorted[k][1] - filtered_tokens_sorted[k+1][1] > count_threshold:
                count_threshold = filtered_tokens_sorted[k][1] - filtered_tokens_sorted[k+1][1]
                actually_upper_bound = filtered_tokens_sorted[k][1]
            if k == len(filtered_tokens_sorted)-2:
                if  filtered_tokens_sorted[k+1][1] > count_threshold:
                        count_threshold = filtered_tokens_sorted[k+1][1]
                        actually_upper_bound =   filtered_tokens_sorted[k+1][1] #including all tokens

        # print(f"count_threshold: {count_threshold}")
        filtered_tokens = dict(filtered_tokens_sorted)
        filtered_tokens = [k[0] for k, v in filtered_tokens.items() if v >= actually_upper_bound]
        
        # Non-DP: Top 5 tokens
        # if len(filtered_tokens_sorted) > 5:
        #     filtered_tokens_sorted = filtered_tokens_sorted[:5]
        # filtered_tokens = [k[0][0] for k in filtered_tokens_sorted]
        # print(filtered_tokens)
        
        # DP: RNM-Gaussian
        
        # ================ End Here ================

        random.shuffle(filtered_tokens) #shuffle the list of tokens
        
        # Build tokens prompt
        suggest_tokens = ""
        for token in filtered_tokens:
            suggest_tokens += token + ", "
        suggest_tokens = suggest_tokens[:-2]
        
        # Build Prompt and generate questions
        icl_prompt = "In the context of the suggested tokens:\n" + suggest_tokens + "\nGenerate a question:"
        question = generate(icl_prompt, temperature=0.0, stop=["\n"])
        
        ## cloud enable section ##
        # Generate answers
        prompt = ""
        for word in paraphrased_df.loc[i]['words']:
            prompt += word + ", "
        prompt = "Extracted OCR tokens from image:\n" + prompt[:-2] + "\nQuestion: " + question + "\nAnswer the question with short term:"
        answers = generate(prompt, temperature=0.0, stop=["\n"])
        
        questions_list.append(question)
        answers_list.append(answers)
    bleu_q_score = bleu_metric.compute(references=questions_gt_list, predictions=questions_list)
    rouge_q_score = rouge_metric.compute(references=questions_gt_list, predictions=questions_list)
    bleu_ans_score = bleu_metric.compute(references=answers_gt_list, predictions=answers_list)
    rouge_ans_score = rouge_metric.compute(references=answers_gt_list, predictions=answers_list)
    bleu_q_scores.append(bleu_q_score['bleu'])
    rouge1_q_scores.append(rouge_q_score['rouge1'])
    rouge2_q_scores.append(rouge_q_score['rouge2'])
    rougeL_q_scores.append(rouge_q_score['rougeL'])
    bleu_a_scores.append(bleu_ans_score['bleu'])
    rouge1_a_scores.append(rouge_ans_score['rouge1'])
    rouge2_a_scores.append(rouge_ans_score['rouge2'])
    rougeL_a_scores.append(rouge_ans_score['rougeL'])

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/UNT/ml1178/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/UNT/ml1178/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


All Tokens:  {('What',): 1, ('NSDA',): 10, ('is',): 5, ('of',): 7, ('address',): 7, ('?',): 10, ('office',): 1, ('location',): 3, ('the',): 8, ('The',): 2, ('for',): 2, ('.',): 1, ('us',): 1, ('Association',): 1, ('Development',): 1, ('write',): 1, ('to',): 1, ('National',): 1, ('question',): 1, ('requires',): 1, ('down',): 1, ('Skills',): 1, ('what',): 1, ('physical',): 1, ('situated',): 1, ('Where',): 4, ('me',): 3, ('provide',): 1, ("'s",): 1, ('you',): 3, ('Can',): 2, ('with',): 1, ('``',): 1, ('Could',): 1, ('tell',): 1, ("''",): 1, ('give',): 1, ('exactly',): 1, ('located',): 2, ('stand',): 1, ('does',): 1}
All Sorted Tokens:  [(('NSDA',), 10), (('?',), 10), (('the',), 8), (('of',), 7), (('address',), 7), (('is',), 5), (('Where',), 4), (('location',), 3), (('me',), 3), (('you',), 3), (('The',), 2), (('for',), 2), (('Can',), 2), (('located',), 2), (('What',), 1), (('office',), 1), (('.',), 1), (('us',), 1), (('Association',), 1), (('Development',), 1), (('write',), 1), (('to',), 1

ValueError: min() arg is an empty sequence

In [57]:
import numpy as np
print(f"BLEU MEAN Question: {np.mean(bleu_q_scores)} STD Question: {np.std(bleu_q_scores)}")
print(f"ROUGE1 MEAN Question: {np.mean(rouge1_q_scores)} STD Question: {np.std(rouge1_q_scores)}")
print(f"ROUGE2 MEAN Question: {np.mean(rouge2_q_scores)} STD Question: {np.std(rouge2_q_scores)}")
print(f"ROUGEL MEAN Question: {np.mean(rougeL_q_scores)} STD Question: {np.std(rougeL_q_scores)}")

print(f"BLEU MEAN Answer: {np.mean(bleu_a_scores)} STD Answer: {np.std(bleu_a_scores)}")
print(f"ROUGE1 MEAN Answer: {np.mean(rouge1_a_scores)} STD Answer: {np.std(rouge1_a_scores)}")
print(f"ROUGE2 MEAN Answer: {np.mean(rouge2_a_scores)} STD Answer: {np.std(rouge2_a_scores)}")
print(f"ROUGEL MEAN Answer: {np.mean(rougeL_a_scores)} STD Answer: {np.std(rougeL_a_scores)}")

BLEU MEAN Question: 0.1403612746310326 STD Question: 0.0050735020516945286
ROUGE1 MEAN Question: 0.4900037151613418 STD Question: 0.0034340205828762484
ROUGE2 MEAN Question: 0.23464050828121655 STD Question: 0.00299716581490447
ROUGEL MEAN Question: 0.44385947177326146 STD Question: 0.0025443003871534436
BLEU MEAN Answer: 0.06455978124121162 STD Answer: 0.009544202964201444
ROUGE1 MEAN Answer: 0.25597692079023654 STD Answer: 0.010747570973046055
ROUGE2 MEAN Answer: 0.14197588944298728 STD Answer: 0.0031398965778053046
ROUGEL MEAN Answer: 0.2570321591379183 STD Answer: 0.01107360938552206
